# Lens Tutorial

In [13]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
{-# LANGUAGE RankNTypes #-}
{-# LANGUAGE FlexibleContexts #-}

import           Data.Functor.Identity
import           Data.Functor.Constant
import           Data.Monoid
import qualified Data.Map as M
import qualified Control.Lens as L
import qualified Control.Lens.Type as L
import           Control.Lens.Operators

data Meetup = Meetup
    { name :: String
    , location :: Location
    } deriving (Show)

data Location = Location
    { latitude :: Latitude
    , longitude :: Longitude
    } deriving (Show)

newtype Latitude = Latitude { getLatitude :: Double } deriving (Show,Num)
newtype Longitude = Longitude { getLongitude :: Double } deriving (Show,Num)

hug :: Meetup
hug = Meetup
 { name = "Frankfurt Haskell Usergroup"
 , location = Location
     { latitude = Latitude 50.1262
     , longitude = Longitude 8.707792
     }
 }
 

("Frankfurt Haskell Usergroup","scala meetup")

In [14]:
mapLongitudeOfMeetup :: (Longitude -> Longitude) -> (Meetup -> Meetup)
mapLongitudeOfMeetup f meetup =
    meetup { location =
              (location meetup) { longitude = 
                  f (longitude (location meetup)) } }

mapLongitudeOfMeetup (+1) hug

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 9.707792}}}

In [16]:
mapLocation :: (Location -> Location) -> (Meetup -> Meetup)
mapLocation f meetup =
    meetup { location = f (location meetup) }

mapLongitude :: (Longitude -> Longitude) -> (Location -> Location)
mapLongitude f loc = loc { longitude = f (longitude loc) }

compose f meetup = mapLocation (mapLongitude f) meetup
:t compose

compose (+1) hug

compose :: (Longitude -> Longitude) -> Meetup -> Meetup

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 9.707792}}}

In [17]:
(mapLocation . mapLongitude) (+1) hug

-- (.) :: (b -> c) -> (a -> b) -> (a -> c)
--
-- Specialized for (mapLocation . mapLongitude)
-- (.) :: ((Location -> Location) -> (Meetup -> Meetup))
--     -> ((Longitude -> Longitude) -> (Location -> Location))
--     -> ((Longitude -> Longitude) -> (Meetup -> Meetup))
:t fmap

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 9.707792}}}

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

In [22]:
mapLocation' :: Functor f => (Location -> f Location) -> (Meetup -> f Meetup)
mapLocation' g meetup = fmap (\loc -> meetup { location = loc }) (g (location meetup))

mapLongitude' :: Functor f => (Longitude -> f Longitude) -> (Location -> f Location)
mapLongitude' f loc = fmap (\long -> loc { longitude = long }) (f (longitude loc))

:t (mapLocation' . mapLongitude')

(mapLocation' . mapLongitude') (\l -> print l >> return (l+1)) hug

(mapLocation' . mapLongitude') :: forall (f :: * -> *). Functor f => (Longitude -> f Longitude) -> Meetup -> f Meetup

Longitude {getLongitude = 8.707792}
Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 9.707792}}}

Identity :: * -> *

Constant :: * -> * -> *

## The Lens Type and Setters

In [20]:
type Lens s t a b = forall f. Functor f => (a -> f b) -> (s -> f t)
type Lens' s a = Lens s s a a

type Setter s t a b = (a -> Identity b) -> s -> Identity t

-- set :: Setter' Meetup Location -> Location -> Meetup -> Meetup
set :: Setter s t a b -> b -> s -> t
set setter b s = runIdentity $ setter (\_ -> Identity b) s 

set (mapLocation' . mapLongitude') 42 hug

set (L.mapped . mapLocation' . mapLongitude') 42 (Just hug)

hug & (mapLocation' . mapLongitude') +~ 1
hug & (mapLocation' . mapLongitude') -~ 1
hug & (mapLocation' . mapLongitude') *~ 2

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 42.0}}}

Just (Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 42.0}}})

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 9.707792}}}

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 7.7077919999999995}}}

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 50.1262}, longitude = Longitude {getLongitude = 17.415584}}}

## Getter

Each lens is a getter

In [24]:
type Getting r s a = (a -> Constant r a) -> (s -> Constant r s)

get :: Getting a s a -> s -> a
get getter s = getConstant $ getter (\a -> Constant a) s


get (mapLocation' . mapLongitude') hug
get (mapLocation' . mapLongitude' . L.to getLongitude) hug

get (L.at 2) (M.fromList [(1,'a'), (2,'b'), (3,'c')])
set (L.at 2) (Just 'c') (M.fromList [(1,'a'), (2,'b'), (3,'c')])
L.at 2 ?~ 'c' $ M.fromList [(1,'a'), (2,'b'), (3,'c')]

Longitude {getLongitude = 8.707792}

8.707792

Just 'b'

fromList [(1,'a'),(2,'c'),(3,'c')]

fromList [(1,'a'),(2,'c'),(3,'c')]

## Traversal

Traversal is weaker than a lens,
i.e. each lens is a traversal but not every traversal is a lens.

In [29]:
mapLongLat :: Applicative f => (Double -> f Double) -> (Location -> f Location)
mapLongLat f loc = Location <$> fmap Latitude (f (getLatitude (latitude loc)))
                            <*> fmap Longitude (f (getLongitude (longitude loc)))

set (mapLocation' . mapLongLat) 4 hug
get (mapLocation' . mapLongLat) hug

Meetup {name = "Frankfurt Haskell Usergroup", location = Location {latitude = Latitude {getLatitude = 4.0}, longitude = Longitude {getLongitude = 4.0}}}

## Fold

In [ ]:
foldMapOf :: Getting r s a -> (a -> r) -> s -> r
foldMapOf lens f s = getConstant $ lens (Constant . f) s

foldrOf :: Getting (Endo r) s a -> (a -> r -> r) -> r -> s -> r
foldrOf l f z = flip appEndo z . foldMapOf l (Endo . f)

toListOf :: Getting (Endo [a]) s a -> s -> [a]
toListOf l = foldrOf l (:) []

toListOf (mapLocation' . mapLongLat) hug

anyOf :: Getting Any s a -> (a -> Bool) -> s -> Bool
anyOf l f = getAny . foldMapOf l (Any . f)

allOf :: Getting All s a -> (a -> Bool) -> s -> Bool
allOf l f = getAll . foldMapOf l (All . f)

anyOf (mapLocation' . mapLongLat) (<= 8) hug
anyOf (mapLocation' . mapLongLat) (<= 9) hug
allOf (mapLocation' . mapLongLat) (>= 8) hug